В данном файле реализуем следующие этапы работы с данными:  
- Создание изображений в соответствии  с полученной от классификатора разметкой.  
- Разбиение датасета из картинок на тренировочную, валидационную и тестовую выборки.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import os
from pathlib import Path
from PIL import Image, ImageDraw
import random
import shutil 
%matplotlib inline

Читаем данные из точки сохранения 2.  
Также сделаем маленькое преобразование с колонкой `dt15`. Дальше будет удобнее, чтобы был формат datetime.

In [2]:
df = pd.read_csv('df_class.csv', encoding='windows-1251', sep=';')
df['dt15'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time15'].astype(str))

### Этап 1. Создаем изображения.  
Будем рисовать изображение для «окна» данных размером 60 строк, т.е. 60 минут. При этом для создания каждого следующего изображения будем сдвигаться вперед на 15-мин интервал и вновь рисовать окно размера 60-минут.

Для хранения наших изображений создадим папки с соответствующими классами.

In [15]:
dir_name = Path('All_pic')
class_names = ['1_buy', '2_nothing', '3_sell']
for class_name in class_names:
    (dir_name / class_name).mkdir(parents=True, exist_ok=True)

На этом этапе рисовать будем изображение цены закрытия, скользящее среднее с периодом 20 и индикатор Болинджера. Сохранять будем изображение 230*228 пикселей в формате RGB.  
Рисовалка `plot_pic` в Matplotlib с пересохранением в RGB. Для нашего датасета будет работать примерно 120 сек - около 2-минут.  

In [16]:
def plot_pic():
    x = np.arange(len(window))
    y = window['Close'].values
    BBup = window['BB_up'].values
    BBlow = window['BB_low'].values
    ma20 = window['ma_20'].values
    # ma40 = window['ma_40'].values
    
    plt.figure(figsize=(2.8, 2.8), dpi=106)
    plt.plot(x, y, color='green')
    plt.plot(x, BBup, color='blue')
    plt.plot(x, BBlow, color='blue')
    plt.plot(x, ma20, color='brown')
    # plt.plot(x, ma40, color='gray')

    ax = plt.gca()
    ax.set_axis_off()
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)

    last_row = window.iloc[-1]
    date_str = last_row['dt15'].strftime('%Y-%m-%d_%H-%M-%S')
    fname = f"SBER-{date_str}.png"
    y_class = int(last_row['Y'])

    class_folder = class_names[y_class - 1]  # Выбираем папку по классу
    plot_path = dir_name / class_folder / fname

    # Сохранение изображения и конвертация в RGB
    plt.savefig(plot_path, bbox_inches='tight', pad_inches=0)
    plt.close('all')

    # Конвертация изображения в RGB и пересохранение
    img = Image.open(plot_path).convert('RGB')
    img.save(plot_path, format="PNG")    

Для класса 2 (около 9600 экз) рисую только около 1000 случайных картинок, чтобы экономить время. Иначе будет рисовать 5-6 минут.

In [11]:
# Стартовая дата (с запасом 60 строк до начала датасета)
# dt15_unique = df[df.datetime >= '2020-01-06 11:00:00']['dt15'].unique()

In [17]:
import matplotlib
matplotlib.use('Agg')
rnd = list(np.random.randint(1, 9600, 1000))                              # Список из 1000 случайных значений (для кл. 2)
counter = 0
period = 60                                                               # для окна размером 60 строк - 60 мин.
dt15_unique = df[df.datetime >= '2020-01-06 11:00:00']['dt15'].unique()   # Старт дата, чт мож было строить окно размера старт - 60
for i in dt15_unique: 
    dt15_index = df.index[df.dt15 == i][0]                                # Берем первый индекс для i из dt15_unique
    window = df.iloc[dt15_index - period: dt15_index]                     # Строим окно размером 60 строк - 60 мин.
    Y_class = int(window.iloc[-1].Y) 

    if Y_class == 2:
        counter += 1
    if Y_class == 1 or Y_class == 3 or counter in rnd:
        plot_pic()                                                        # Вызываем нашу функцию для рисования.


Этап 1 на этом завершен. Далее созданные картинки надо разделить на трейн, валидацию и тест.

### Этап 2. Разделение выборки на train, validation, test

Сначала глянем сколько имеется файлов каждого класса. Найдем минимум. Исходя из него прикинем как разделить выборку.
Пусть пропорция для train/validation/test будет такой: 70/15/15.

In [18]:
data_root = 'All_pic'
c = []
for i in os.listdir(data_root):
    dir = os.path.join(data_root, i)
    c.append(len(os.listdir(dir)))
c_min = min(c)
tr = round(c_min / 100 * 70)
tst = round(c_min / 100 * 15)
vld = round(c_min / 100 * 15)
print(c)
print(f"train:{tr}, test:{tst}, validation:{vld}, sum_all:{sum([tr, tst, vld])}")

[753, 952, 789]
train:527, test:113, validation:113, sum_all:753


Теперь создадим новую папку `Pic_for_CNN` с нужным количеством картинок для train, validation, test.  
Тут пока **вручную** пропишем кол-во. Пусть оно будет четным, для более удобного деления на батчи, на этапе обучения нейросети.  
`split_counts = {'train': 526, 'validation': 112, 'test': 112}`. Случайным образом отберем картинки в нужные директории.

In [19]:
# Пути к исходной и целевой директории
src_dir = 'All_pic'
dest_dir = 'Pic_for_CNN'

# Категории классов
categories = ['1_buy', '2_nothing', '3_sell']

# Количество изображений для каждого набора
split_counts = {'train': 526, 'validation': 112, 'test': 112}

# Создание целевой структуры папок
for split in ['train', 'validation', 'test']:
    for category in categories:
        os.makedirs(os.path.join(dest_dir, split, category), exist_ok=True)

# Разделение изображений
for category in categories:
    # Получаем список всех изображений в папке
    img_list = os.listdir(os.path.join(src_dir, category))
    
    # Перемешиваем изображения для случайного выбора
    random.shuffle(img_list)
    
    start = 0
    for split, count in split_counts.items():
        selected_images = img_list[start:start + count]
        start += count
        
        for img in selected_images:
            src_path = os.path.join(src_dir, category, img)
            dest_path = os.path.join(dest_dir, split, category, img)
            
            shutil.copy(src_path, dest_path)  

print("Готово!")

Готово!


Этап 2 на этом завершен. Теперь у нас имеется полностью готовый датасет из картинок, на котором мы будем пытаться обучить нейросеть. Сделаем это в новом файле - `PatternCNN_colab`.